In [1]:
import pandas as pd
from openpyxl import load_workbook
import os
import pandas as pd
import numpy as np

In [2]:
data_path = '..\\datasets'
files = os.listdir(data_path)
files.remove('URL descripcion reto.txt')
files.remove('leeme.txt')
dfs = {}

def open_file(file_name):
    file_path = os.path.join(data_path, file_name)

    # Cargar todas las hojas con pandas.read_excel(sheet_name=None)
    all_sheets = pd.read_excel(file_path, sheet_name=None)

    # Guardar cada hoja en dfs con un nombre único combinando el archivo y la hoja
    for sheet_name, data in all_sheets.items():
        # Crear un nombre único para el DataFrame usando el nombre del archivo y la hoja
        df_name = f"{file_name.replace('.xlsx', '')}_{sheet_name}"
        dfs[df_name] = data


# Abrir todos los archivos y leer todas las hojas
for file in files:
    if file.endswith('.xlsx'):  # Asegurarse de que solo se procesen archivos Excel
        open_file(file)


In [3]:
print(dfs.keys())

dict_keys(['Biorreactor 13169_WData', 'Biorreactor 13169_Datos', 'Biorreactor 13170_WData', 'Biorreactor 13170_Datos', 'Biorreactor 13171_WData', 'Biorreactor 13171_Datos', 'Biorreactor 13172_WData', 'Biorreactor 13172_Datos', 'Biorreactor 14614_Datos', 'Biorreactor 14615_WData', 'Biorreactor 14615_Datos', 'Biorreactor 14616_WData', 'Biorreactor 14616_Datos', 'Biorreactor 14617_WData', 'Biorreactor 14617_Datos', 'Biorreactor 14618_WData', 'Biorreactor 14618_Datos', 'Centrífuga 12912_WData', 'Centrífuga 12912_Datos', 'Centrífuga 14246_WData', 'Centrífuga 14246_Datos', 'Centrífuga 17825_WData', 'Centrífuga 17825_Datos', 'Cinéticos IPC_Inóculos', 'Cinéticos IPC_Cultivos finales', 'Cinéticos IPC_Centrifugación', 'Fases producción v02_Preinóculo', 'Fases producción v02_Inóculo', 'Fases producción v02_Cultivo final', 'Fases producción v03 Test_Cultivo final', 'Fases producción v03_Preinóculo', 'Fases producción v03_Inóculo', 'Fases producción v03_Cultivo final', 'Fases producción_test v02_Cu

In [4]:
# Acceder a los DataFrames

df_preinoculo = dfs['Fases producción v03_Preinóculo']
df_preinoculo.columns = df_preinoculo.iloc[0]
df_preinoculo = df_preinoculo.drop(0)
df_inoculo = dfs['Fases producción v03_Inóculo']
df_cultivo_final = dfs['Fases producción v03_Cultivo final']

merged_df = df_preinoculo.merge(df_inoculo, on='LOTE', suffixes=('_preinoculo', '_inoculo'))
merged_df = merged_df.merge(df_cultivo_final, on='LOTE', suffixes=('', '_cultivo_final'))

In [5]:
import numpy as np

In [6]:
def load_centrifuga_data(id_centrifuga):
    file_name = f'Centrífuga {id_centrifuga}_Datos'
    try:
        # Intentar cargar el DataFrame de centrífuga
        centrifuga_df = dfs[file_name]
        return centrifuga_df
    except KeyError:
        # Si el archivo no existe, retornar un DataFrame vacío o NaN
        return pd.DataFrame()  # o simplemente return np.nan si prefieres


def calculate_mean_for_lote(lote, fecha_inicio, fecha_fin, id_centrifuga):
    centrifuga_df = load_centrifuga_data(id_centrifuga)
    if centrifuga_df.empty:
        return np.nan  # Si no hay datos, devolver NaN
    mask = (pd.to_datetime(centrifuga_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & (pd.to_datetime(centrifuga_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = centrifuga_df[mask]
    return filtered_data[f'{id_centrifuga}_D01916047.PV'].mean()

merged_df['media_PV'] = merged_df.apply(
    lambda row: calculate_mean_for_lote(row['LOTE'], row['Fecha/hora inicio'], row['Fecha/hora fin'], row['ID Centrífuga']),
    axis=1
)

In [7]:



# Function to load bioreactor data based on the given ID
def load_bioreactor_data(id_bioreactor):
    file_name = f'Biorreactor {id_bioreactor}_Datos'
    try:
        # Attempt to load the DataFrame of the bioreactor
        bioreactor_df = dfs[file_name]
        return bioreactor_df
    except KeyError:
        # If the file does not exist, return an empty DataFrame
        return pd.DataFrame()  # Alternatively, return np.nan if preferred

# Function to calculate the mean for the specified lot and bioreactor
def calculate_mean_for_lote2(lote, fecha_inicio, fecha_fin, id_bioreactor):
    bioreactor_df = load_bioreactor_data(id_bioreactor)
    
    if bioreactor_df.empty:
        return np.nan, np.nan, np.nan  # Return NaN for all means if there's no data

    # Filter the data based on the specified date range
    mask = (pd.to_datetime(bioreactor_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & \
           (pd.to_datetime(bioreactor_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = bioreactor_df[mask]
    
    # Calculate means for the specified columns
    mean_temp = filtered_data[f'{id_bioreactor}_FERM0101.Temperatura_PV'].mean() if not filtered_data.empty else np.nan
    mean_ph = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_pH_PV'].mean() if not filtered_data.empty else np.nan
    mean_do = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_DO_PV'].mean() if not filtered_data.empty else np.nan
    
    return mean_temp, mean_ph, mean_do

# Apply the function to each row of merged_df and store the results in new columns
merged_df[['media_temp_bioreactor', 'media_ph_biorreactor', 'media_PO_biorreactor']] = merged_df.apply(
    lambda row: calculate_mean_for_lote2(row['LOTE'], row['Fecha/hora inicio_inoculo'], row['Fecha/hora fin_inoculo'], row['ID bioreactor']),
    axis=1,
    result_type='expand'  # This allows unpacking into multiple columns
)

In [8]:
merged_df = merged_df.fillna("NA")
output_file_path = 'dataset_fase1.csv'  # Cambia esto a la ruta donde quieras guardar el archivo
merged_df.to_csv(output_file_path, index=False)

In [9]:
merged_df.c

AttributeError: 'DataFrame' object has no attribute 'c'